# SCohenLab 2D BATCH Image Processing notebook (Simplified MCZ)

--------------
# PIPELINE OVERVIEW
## ❶ GOAL SETTING ✍

### GOAL:  Infer sub-cellular components in order to understand interactome 

To measure shape, position, size, and interaction of eight organelles/cellular components (Nuclei (NU), Lysosomes (LS),Mitochondria (MT), Golgi (GL), Peroxisomes (PO), Endoplasmic Reticulum (ER), Lipid Droplet (LD), and SOMA) during differentiation of iPSCs, in order to understand the Interactome / Spatiotemporal coordination.

### summary of _OBJECTIVES_ ✅
- robust inference of subcellular objects:
  + 1️⃣-***nuclei***
  + 2️⃣-***cellmask***
  + 3️⃣-***cytoplasm*** (+ ***nucleus***)
  + 4️⃣-***lysosome***
  + 5️⃣-***mitochondria***
  + 6️⃣-***golgi***
  + 7️⃣-***peroxisome***
  + 8️⃣-***ER***
  + 9️⃣-***lipid droplet***





## ❷ DATA CREATION
> METHODS:📚📚
> 
> iPSC lines prepared and visualized on Zeiss Microscopes. 32 channel multispectral images collected.  Linear Unmixing in  ZEN Blue software with target emission spectra yields 8 channel image outputs.  Channels correspond to: Nuclei (NU), Lysosomes (LS),Mitochondria (MT), Golgi (GL), Peroxisomes (PO), Endoplasmic Reticulum (ER), Lipid Droplet (LD), and a “residual” signal.

> Meta-DATA 🏺 (artifacts)
>   - Microcope settings
>  - OME scheme
> - Experimenter observations
> - Sample, bio-replicate, image numbers, condition values, etc
>  - Dates
>  - File structure, naming conventions
>  - etc.





## ❸. IMAGE PROCESSING ⚙️🩻🔬
### INFERENCE OF SUB-CELLULAR OBJECTS
The imported images have already been pre-processed to transform the 32 channel spectral measuremnts into "linearly unmixed" images which estimate independently labeled sub-cellular components.  Thes 7 channels (plus a residual "non-linear" signal) will be used to infer the shapes and extents of these sub-cellular components.   
We will perform computational image analysis on the pictures (in 2D an 3D) to _segment_ the components of interest for measurement.  In other prcoedures we can used these labels as "ground truth" labels to train machine learning models to automatically perform the inference of these objects.
Pseudo-independent processing of the imported multi-channel image to acheive each of the 9 objecives stated above.  i.e. infering: NUCLEI, SOMA, CYTOSOL, LYSOSOME, MITOCHONDRIA, GOLGI COMPLEX, PEROZISOMES, ENDOPLASMIC RETICULUM, and LIPID BODIES

### General flow for infering objects via segmentation
- Pre-processing 🌒
- Core-processing (thresholding) 🌕
- Post-processing  🌘

### QC 🚧 WIP 🚧 




## ❹. QUANTIFICATION 📏📐🧮

SUBCELLULAR COMPONENT METRICS
-  extent 
-  size
-  shape
-  position



### NOTE: PIPELINE TOOL AND DESIGN CHOICES?
We want to leverage the Allen Cell & Structure Setmenter.  It has been wrapped as a [napari-plugin](https://www.napari-hub.org/plugins/napari-allencell-segmenter) but fore the workflow we are proving out here we will want to call the `aicssegmentation` [package](https://github.com/AllenCell/aics-segmentation) directly.

#### ​The Allen Cell & Structure Segmenter 
​The Allen Cell & Structure Segmenter is a Python-based open source toolkit developed at the Allen Institute for Cell Science for 3D segmentation of intracellular structures in fluorescence microscope images. This toolkit brings together classic image segmentation and iterative deep learning workflows first to generate initial high-quality 3D intracellular structure segmentations and then to easily curate these results to generate the ground truths for building robust and accurate deep learning models. The toolkit takes advantage of the high replicate 3D live cell image data collected at the Allen Institute for Cell Science of over 30 endogenous fluorescently tagged human induced pluripotent stem cell (hiPSC) lines. Each cell line represents a different intracellular structure with one or more distinct localization patterns within undifferentiated hiPS cells and hiPSC-derived cardiomyocytes.

More details about Segmenter can be found at https://allencell.org/segmenter
In order to leverage the A




## IMPORTS

In [ ]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict
from typing import Optional, Union

import numpy as np
import pandas as pd
from skimage.measure import regionprops_table, regionprops

from aicssegmentation.core.pre_processing_utils import  image_smoothing_gaussian_slice_by_slice 
from skimage.measure import label

import napari

### import local python functions in ../infer_subc_2d
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc_2d.core.file_io import (read_czi_image,
                                                                    export_inferred_organelle,
                                                                    list_image_files)

from infer_subc_2d.core.img import *
from infer_subc_2d.organelles import * 

%load_ext autoreload
%autoreload 2



In [ ]:
# NOTE:  these "constants" are only accurate for the testing MCZ dataset
from infer_subc_2d.constants import (TEST_IMG_N,
                                                                    NUC_CH ,
                                                                    LYSO_CH ,
                                                                    MITO_CH ,
                                                                    GOLGI_CH ,
                                                                    PEROX_CH ,
                                                                    ER_CH ,
                                                                    LD_CH ,
                                                                    RESIDUAL_CH )              


## SETUP

In [ ]:
# this will be the example image for testing the pipeline below
test_img_n = TEST_IMG_N

# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
in_data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(in_data_path,im_type)
test_img_name = img_file_list[test_img_n]

# save output ".tiff" files here
out_data_path = data_root_path / "out"

if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

In [ ]:
img_data,meta_dict = read_czi_image(test_img_name)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']


In [ ]:
scale[0]/scale[1], scale

Now get the single "optimal" slice of all our organelle channels....

## get the inferred cellmask, nuclei and cytoplasm objects

(takes < 1 sec)

Builde the segmentations in order




In [ ]:
###################
# SOMA, NUCLEI, CYTOSOL, NUCLEUS
###################
nuclei_obj =  infer_nuclei_fromlabel(img_data,meta_dict, out_data_path)
soma_obj  = get_soma(img_data, nuclei,meta_dict, out_data_path)
cytoplasm_mask = get_cytoplasm(nuclei_obj , soma_obj , meta_dict, out_data_path)
# get overall summary stats for cellmask


In [ ]:
# names of organelles we have
organelle_names = ["lyso", "mitochondria","golgi","peroxisome","er","lipid"]

get_methods  = [get_lyso,
            get_mito,
            get_golgi,
            get_perox,
            get_ER,
            get_LD]

# load all the organelle segmentations
organelles = [meth(img_data,meta_dict, out_data_path) for meth in get_methods]

# get the intensities
organelle_channels = [LYSO_CH,MITO_CH,GOLGI_CH,PEROX_CH,ER_CH,LD_CH]

intensities = [img_data[ch] for ch in organelle_channels]


In [ ]:
organelle = 1
mask = cytoplasm_mask
segmentation = np.stack([
                            nuclei_object,
                            lyso_object,
                            mito_object,
                            golgi_object,
                            peroxi_object,
                            er_object,
                            LD_object ])

florescence = img_2D[:-1] # throw out residual


florescence.shape, segmentation.shape, mask.shape


In [ ]:

# CHOOSE which inferred organelle to 
if organelle == NUC_CH: #0
    organelle_name = 'nucleus'
elif organelle == LYSO_CH: #1 
    organelle_name = 'lyso'
elif organelle == MITO_CH: #2
    organelle_name = 'mitochondria'
elif organelle == GOLGI_CH: #3
    organelle_name = 'golgi'
elif organelle == PEROX_CH: #4
    organelle_name = 'peroxisome'
elif organelle == ER_CH: #5
    organelle_name = 'ER'
elif organelle == LD_CH: #6 
    organelle_name = 'lipid'
elif organelle == RESIDUAL_CH: #7
    organelle_name = 'residual'

print(organelle_name)

target = segmentation[organelle].squeeze()
target = apply_mask(target, mask.squeeze())    
target_intensity = florescence[organelle].squeeze()

labels = label(target).astype("int").squeeze()


summary_stats = collect_organelle_stats(
                                target, 
                                target_intensity
                                )
# target.shape, target_intensity.shape, labels.shape

labels.max()



## Visualize with `napari` 1
Visualize the first-pass segmentation and labeling with `napari`.

In [ ]:
viewer = napari.Viewer()

In [ ]:
viewer.add_image(
    nuclei_object,
    scale=scale,
    colormap='blue', 
    blending='additive'
)
viewer.add_image(
    lyso_object,
    scale=scale,
    colormap='cyan', 
    blending='additive'
)

viewer.add_image(
    mito_object,
    scale=scale,
    colormap='green', 
    blending='additive'
)

viewer.add_image(
    golgi_object,
    scale=scale,
    colormap='yellow', 
    blending='additive'
)


viewer.add_image(
    peroxi_object,
    scale=scale,
    colormap='bop orange', 
    blending='additive'
)


viewer.add_image(
    er_object,
    scale=scale,
    blending='additive')

viewer.add_image(
    LD_object,
    scale=scale,
    blending='additive')



In [ ]:
viewer.scale_bar.visible = True
from napari.utils.notebook_display import nbscreenshot

# viewer.dims.ndisplay = 3
# viewer.camera.angles = (-30, 25, 120)
nbscreenshot(viewer, canvas_only=True)

In [ ]:
viewer.close()


There may be a bug where the input images to the "infer_*" functions are modified in place and we might need to access them.  _MASKING_ seems to be the problem.  Also need to be clear about _when_ to apply the mask.

In [ ]:
from infer_subc_2d.core.img import select_z_from_raw
###########
# infer organelles
##########
def _fixed_infer_organelles(img_data):
    """
    wrapper to infer all organelles from a single multi-channel image
    """
    # ch_to_agg = (LYSO_CH, MITO_CH, GOLGI_CH, PEROX_CH, ER_CH, LD_CH)

    # nuc_ch = NUC_CH
    # optimal_Z = find_optimal_Z(img_data, nuc_ch, ch_to_agg)
    # # Stage 1:  nuclei, cellmask, cytoplasm
    # img_2D = select_z_from_raw(img_data, optimal_Z)
    img_2D = fixed_get_optimal_Z_image(img_data)

    soma_mask = fixed_infer_cellmask_fromaggr(img_2D)

    nuclei_object = fixed_infer_nuclei(img_2D, soma_mask)

    cytoplasm_mask = infer_cytoplasm(nuclei_object, soma_mask)

    # cyto masked objects.
    lyso_object = fixed_infer_lyso(img_2D, cytoplasm_mask)
    mito_object = fixed_infer_mito(img_2D, cytoplasm_mask)
    golgi_object = fixed_infer_golgi(img_2D, cytoplasm_mask)
    peroxi_object = fixed_infer_perox(img_2D, cytoplasm_mask)
    er_object = fixed_infer_ER(img_2D, cytoplasm_mask)
    LD_object = fixed_infer_LD(img_2D, cytoplasm_mask)

    img_layers = [
        nuclei_object,
        lyso_object,
        mito_object,
        golgi_object,
        peroxi_object,
        er_object,
        LD_object,
        soma_mask,
        cytoplasm_mask,
    ]

    layer_names = [
        "nuclei",
        "lyso",
        "mitochondria",
        "golgi",
        "peroxisome",
        "er",
        "LD_body",
        "soma_mask",
        "cytoplasm_mask",
    ]
    # TODO: pack outputs into something napari readable
    img_out = np.stack(img_layers, axis=0)
    return (img_out, layer_names, optimal_Z)


In [ ]:

def stack_organelle_objects(soma_mask,
                            nuclei_object,
                            cytoplasm_mask,
                            lyso_object,
                            mito_object,
                            golgi_object,
                            peroxi_object,
                            er_object,
                            LD_object) -> np.ndarray:
    """ wrapper to stack the inferred objects into a single numpy.ndimage """
    img_layers = [soma_mask,
                            nuclei_object,
                            cytoplasm_mask,
                            lyso_object,
                            mito_object,
                            golgi_object,
                            peroxi_object,
                            er_object,
                            LD_object]
    return np.stack(img_layers, axis=0) 

In [ ]:
def stack_organelle_layers(*layers) -> np.ndarray:
    """ wrapper to stack the inferred objects into a single numpy.ndimage """

    return np.stack(layers, axis=0) 

In [ ]:
stacked_organelle_objects(*img_layers).shape

In [ ]:

inferred_organelles, layer_names, optimal_Z = _fixed_infer_organelles(img_data)


In [ ]:
viewer = napari.Viewer()


In [ ]:
cmaps = ['blue','cyan','green','yellow','bop orange','magenta','gray','gray','gray']

for i,organelle in enumerate(inferred_organelles):
    viewer.add_image(
        organelle,
        scale=scale,
        blending='additive',
        name = layer_names[i],
        colormap=cmaps[i]
    )


In [ ]:
viewer.scale_bar.visible = True

nbscreenshot(viewer, canvas_only=True)

In [ ]:
meta_dict

In [ ]:

##################
# export
##################

def _export_infer_organelles(img_out, layer_names, meta_dict, data_root_path):
       # get some top-level info about the RAW data
    # channel_names = meta_dict['name']
    # img = meta_dict['metadata']['aicsimage']
    # scale = meta_dict['scale']
    # channel_axis = meta_dict['channel_axis']
    img_name = meta_dict['file_name']
    # add params to metadata
    meta_dict['layer_names'] = layer_names
    out_path = data_root_path / "inferred_objects" 
    img_name_out = 'binarized_' + img_name.split("/")[-1].split(".")[0]

    out_file_n = export_ome_tiff(img_out, meta_dict, img_name_out, str(out_path)+"/", layer_names)
    print(f"saved file: {out_file_n}")
    return out_file_n

out_file_n = _export_infer_organelles(inferred_organelles, layer_names, meta_dict, data_root_path)



In [ ]:
out_file_n

----------------------

run a batch of ALL the images

First get all the images

In [ ]:
# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"



now build a function to loop over them all and export




In [ ]:

def batch_process_all_czi(data_root_path):

    # linearly unmixed ".czi" files are here
    data_path = data_root_path / "raw"
    im_type = ".czi"
    # get the list of all files
    img_file_list = list_image_files(data_path,im_type)
    files_generated = []
    for czi_file in img_file_list:
        out_fn = process_czi_image(czi_file)
        files_generated.append(out_fn)

    print(f"generated {len(files_generated)} ")
    return files_generated

def process_czi_image(czi_file_name):
    """wrapper for processing"""

    img_data,meta_dict = read_czi_image(czi_file_name)
    # # get some top-level info about the RAW data
    # channel_names = meta_dict['name']
    # img = meta_dict['metadata']['aicsimage']
    # scale = meta_dict['scale']
    # channel_axis = meta_dict['channel_axis']

    inferred_organelles, layer_names,optimal_Z = _infer_organelles(img_data)
    meta_dict['z_slice'] = optimal_Z
    out_file_n = _export_infer_organelles(inferred_organelles, layer_names, meta_dict, data_root_path)

    ## TODO:  collect stats... 

    return out_file_n

# chan_name = 'nuclei'
# out_path = data_root_path / "inferred_objects" 
# object_name = 'NU_object'

# NU_bioim = read_input_image( out_path/ f"{object_name}.ome.tiff"  )
# NU_object = NU_bioim.image
# NU_labels = label(NU_object)

# # calculate a filter dimension for median filtering which considers the difference in scale of Z
# z_factor = scale[0]//scale[1]
# med_filter_size = 4 #2D 
# med_filter_size_3D = (1,med_filter_size,med_filter_size)  # set the scale for a typical median filter

In [ ]:
output_tiffs = batch_process_all_czi(data_root_path)


Write the `infer_cellmask_fromaggr` spec to the widget json

In [ ]:
from infer_subc_2d.organelles_config.helper import add_function_spec_to_widget_json

_fixed_infer_organelles =  {
        "name": "infer all organelles (fixed parameters)",
        "python::module": "infer_subc_2d.batch.batch_process",
        "python::function": "fixed_infer_organelles",
        "parameters": None
        }

add_function_spec_to_widget_json("fixed_infer_organelles",_fixed_infer_organelles)

In [ ]:

_stack_organelle_objects =  {
        "name": "stack organelles, argv spelled out",
        "python::module": "infer_subc_2d.batch",
        "python::function": "stack_organelle_objects",
        "parameters": None
        }

add_function_spec_to_widget_json("stack_organelle_objects",_stack_organelle_objects)


_stack_organelle_layers =  {
        "name": "stack organelles, *argv ",
        "python::module": "infer_subc_2d.batch",
        "python::function": "stack_organelle_layers",
        "parameters": None
        }

add_function_spec_to_widget_json("stack_organelle_layers",_stack_organelle_layers)


-------------------------------
## Write workflow .json
Now that we've added our function specs we can compose workflows.

In [ ]:

def make_fixed_infer_organelles_batch_dict():
    """
    Procedure to infer mitochondria from linearly unmixed input from raw

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
   
    ###################
    # Stage 1:  nuclei, cellmask, cytoplasm
    ###################   
    step_name.append("1")
    function_name.append("fixed_get_optimal_Z_img")
    category.append("extraction")
    parameter_values.append(None)
    parent.append(0)

    step_name.append("2")
    function_name.append("fixed_infer_cellmask_fromaggr")
    category.append("core")
    parameter_values.append( None )
    parent.append(1)

    step_name.append("3")
    function_name.append("fixed_infer_nuclei")
    category.append("core")
    parameter_values.append( None )
    parent.append([1,2])

    step_name.append("4")
    function_name.append("infer_cytoplasm")
    category.append("core")
    parameter_values.append(dict(erode_nuclei = True ))
    parent.append([2,3])

    ###################
    # Stage 2:  cyto masked objects
    ###################   
    step_name.append("5")
    function_name.append("fixed_infer_lyso")
    category.append("core")
    parameter_values.append(None )
    parent.append([1,4])

    
    step_name.append("6")
    function_name.append("fixed_infer_mito")
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("7")
    function_name.append("fixed_infer_golgi")
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("8")
    function_name.append("fixed_infer_perox")
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("9")
    function_name.append("fixed_infer_ER") 
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("10")
    function_name.append("fixed_infer_LD") 
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("11")
    function_name.append("stack_organelle_objects") 
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append([2,3,4,5,6,7,8,9,10])


    # TODO: add export functions

    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
    
    return out_dict



In [ ]:
from infer_subc_2d.organelles_config.helper import write_workflow_json

infer_fixed_infer_organelles_batch_dict = make_fixed_infer_organelles_batch_dict()

write_workflow_json("infer_fixed_infer_organelles_batch", infer_fixed_infer_organelles_batch_dict)

In [ ]:

def make_fixed_infer_organelles_batch_dict2():
    """
    Procedure to infer mitochondria from linearly unmixed input from raw
    """
    
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
   
    ###################
    # Stage 1:  nuclei, cellmask, cytoplasm
    ###################   
    step_name.append("1")
    function_name.append("fixed_get_optimal_Z_img")
    category.append("extraction")
    parameter_values.append(None)
    parent.append(0)

    step_name.append("2")
    function_name.append("fixed_infer_cellmask_fromaggr")
    category.append("core")
    parameter_values.append( None )
    parent.append(1)

    step_name.append("3")
    function_name.append("fixed_infer_nuclei")
    category.append("core")
    parameter_values.append( None )
    parent.append([1,2])

    step_name.append("4")
    function_name.append("infer_cytoplasm")
    category.append("core")
    parameter_values.append(dict(erode_nuclei = True ))
    parent.append([2,3])

    ###################
    # Stage 2:  cyto masked objects
    ###################   
    step_name.append("5")
    function_name.append("fixed_infer_lyso")
    category.append("core")
    parameter_values.append(None )
    parent.append([1,4])

    
    step_name.append("6")
    function_name.append("fixed_infer_mito")
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("7")
    function_name.append("fixed_infer_golgi")
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("8")
    function_name.append("fixed_infer_perox")
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("9")
    function_name.append("fixed_infer_ER") 
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("10")
    function_name.append("fixed_infer_LD") 
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])


    step_name.append("11")
    function_name.append("stack_organelle_objects") 
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append([2,3,4,5,6,7,8,9,10])

    # TODO: add export functions

    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
    
    return out_dict



In [ ]:
from infer_subc_2d.organelles_config.helper import write_workflow_json

infer_fixed_infer_organelles_batch_dict2 = make_fixed_infer_organelles_batch_dict2()

write_workflow_json("infer_fixed_infer_organelles_batch2", infer_fixed_infer_organelles_batch_dict2)

In [ ]:
#TODO: make infer_organelles function with the exhaustive list of parmaters
_infer_organelles =  {
        "name": "Infer Endoplasmic Reticulum",
        "python::module": "infer_subc_2d.organelles",
        "python::function": "infer_ER",
        "parameters": {
                "filament_scale": {
                        "data_type": "float",
                        "increment": 0.05,
                        "max": 10,
                        "min": 0,
                        "widget_type": "slider"
                },
                "filament_cut": {
                        "data_type": "float",
                        "increment": 0.001,
                        "max": 0.5,
                        "min": 0,
                        "widget_type": "slider"
                },
                "small_obj_w": {
                        "data_type": "int",
                        "increment": 1,
                        "max": 50,
                        "min": 1,
                        "widget_type": "slider"
                }
        }
}

add_function_spec_to_widget_json("eeeinfer_organelles", _infer_organelles, overwrite=True )



35 files processed in 6 minutes 47 seconds!!

In [ ]:

tiff_img_data,tiff_meta_dict = read_czi_image(output_tiffs[-1])


In [ ]:
tiff_img_data.shape

img = tiff_meta_dict['metadata']['aicsimage']
img.dims

In [ ]:
from aicsimageio.writers import OmeTiffWriter
data_in = img_out
channel_names = [layer_names]
image_names = [img_name]
print(image_names)
# chan_names = meta_in['metadata']['aicsimage'].channel_names
dimension_order = ["CZYX"]

num_images = len(  [data_in.shape])
if data_in.dtype == "bool":
    data_in = data_in.astype(np.uint8)
    data_in[data_in > 0] = 255

physical_pixel_sizes = [meta_dict["metadata"]["aicsimage"].physical_pixel_sizes]
out_ome = OmeTiffWriter.build_ome(
        [data_in.shape],
        [data_in.dtype],
        channel_names=channel_names,  # type: ignore
        image_name=image_names,
        physical_pixel_sizes=physical_pixel_sizes,
        dimension_order=dimension_order,
    )